# deforesation quantification using classification based object detection 


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os, pathlib

import numpy as np
import time
import datetime
import math


import PIL.Image as Image
import matplotlib.pylab as plt
import matplotlib as mpl


os.environ['TF_USE_LEGACY_KERAS'] = '1'
import tensorflow as tf
import tensorflow_hub as hub

# https://github.com/tensorflow/tensorflow/issues/65419
# import tf_keras
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tf_keras as keras
else:
    keras = tf.keras

from tensorflow.keras import layers

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
tf.config.experimental.set_memory_growth(gpus[0], enable=True)

In [ ]:
input_data_dir = './../../Data/MultiModalGenAI/resisc45/NWPU-RESISC45_small'
output_models_dir = './../../models/MultiModalGenAI/deforestation'
modelfname= 'deforestation_model_1735604427'

batch_size = 32
img_height = 224
img_width = 224


In [ ]:
trained_model = tf.keras.models.load_model(f'{output_models_dir}/{modelfname}')

In [ ]:
dataset_path = pathlib.Path(input_data_dir).with_suffix('')
image_count = len(list(dataset_path.glob('*/*.jpg')))
print(image_count)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  str(dataset_path),
  validation_split=0.4,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(dataset_path),
  validation_split=0.4,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for val_image_batch, val_labels_batch in val_ds:
  print(val_image_batch.shape)
  print(val_image_batch.shape)
  break

In [ ]:
val_predictions_batch = trained_model.predict(val_image_batch)

In [ ]:
val_predicted_id = tf.math.argmax(val_predictions_batch, axis=-1)
val_predicted_label_batch = class_names[val_predicted_id]
print(val_predicted_label_batch)


In [ ]:
_ = plt.figure(figsize=(10,12))
_ = plt.subplots_adjust(wspace=.5, hspace=0.5)
for n in range(30):
    _ = plt.subplot(5,6,n+1)
    _ = plt.imshow(val_image_batch[n])
    _ = plt.title(f'Pred:{str(val_predicted_label_batch[n].title())}\nReal:{class_names[val_labels_batch[n]]}')
    _ = plt.axis('off')
    _ = plt.suptitle("Model predictions on Validation Data")

## Patched Object Detection using transfer learning based image classification 


In [ ]:
!pwd
!ls -la ./../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/Deforestation10*

In [ ]:
get_patches = lambda x: (tf.reshape(
    tf.image.extract_patches(
        images=tf.expand_dims(x, 0),
        sizes=[1, patch_height, patch_width, 1],
        strides=[1, stride_height, stride_width, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'), (-1, patch_height, patch_width, 3))
)

def create_heatmap(predicts_ids, patches_order, patches_shape):
    patch_num_y, patch_num_x = patches_order
    patch_height, patch_width = patches_shape
    # heatmap = np.full(patches_shape, np.nan)
    index = 0
    heatmap = np.full(patches_shape, predicts_ids[index])
    index+=1
    
    for col_index in range(patch_num_x-1):
        heatmap = np.concatenate((heatmap,np.full(patches_shape, predicts_ids[index])),axis=1)
        index+=1
        
    for row_index in range(patch_num_y-1):
        rowarray  = np.full(patches_shape, predicts_ids[index])
        index+=1
        for col_index in range(patch_num_x-1):
            rowarray = np.concatenate((rowarray,np.full(patches_shape, predicts_ids[index])),axis=1)
            index+=1
        heatmap = np.concatenate((heatmap, rowarray),axis=0)
        
    return heatmap

# https://keras.io/examples/vision/grad_cam/
def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Use jet colormap to colorize heatmap
    jet = mpl.colormaps["jet"]

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    # jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image, cropped to match the heatmap
    superimposed_img = jet_heatmap * alpha + img[:heatmap.shape[0],:heatmap.shape[1],:]
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    return  tf.keras.utils.load_img(cam_path)


resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(img_height, img_width),
  layers.Rescaling(1./255)
])

In [ ]:
patch_width = math.floor(img_width*1)
patch_height = math.floor(img_height*1)
stride_width = patch_width
stride_height = patch_height

In [ ]:
# image_f = 'Deforestation10_after_12_2020.jpg'
# image_f = 'Deforestation10_before_12_1999.jpg'
# image_f = 'Deforestation05_after_12_2019.jpg'
# image_f = 'Deforestation05_before_12_2018.jpg'
image_f = 'Deforestation01_after_06_2012.jpg'

images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'
target_img = tf.keras.utils.load_img(images_path+image_f)
target_arr  = tf.keras.utils.img_to_array(target_img)
target_arr.shape

In [ ]:
patches = get_patches(target_arr)
patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)


In [ ]:
_=fig = plt.figure(figsize=(15,10))
_=plt.subplots_adjust(wspace=.05, hspace=.05)

for index in range(patches.shape[0]):
    arr_ = np.squeeze(patches[index,])
    ax = plt.subplot(patch_num_y, patch_num_x,  index + 1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(arr_/255)
_=plt.show()

In [ ]:
patches_predicts = trained_model.predict(resize_and_rescale(patches))
patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
patches_predicts_label = class_names[patches_predicts_id]
patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

# classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

save_and_display_gradcam(target_arr, classifHeatmapNorm)
keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2))
keras.utils.array_to_img(target_arr)


patches.shape
(patch_width, patch_height)
(stride_width, stride_height)
(patch_num_y, patch_num_x)

print(patches_predicts_label)
print(patches_predicts_id)

classifHeatmap.shape
classifHeatmapNorm.shape

In [ ]:
# image_f = 'Deforestation01_before_07_2011.jpg'
image_f = 'Deforestation03HR_after_09_2019.jpg'

images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'
target_img = tf.keras.utils.load_img(images_path+image_f)
target_arr  = tf.keras.utils.img_to_array(target_img)

patches = get_patches(target_arr)
patches_predicts = trained_model.predict(resize_and_rescale(patches))
patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
patches_predicts_label = class_names[patches_predicts_id]
patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)
# classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

save_and_display_gradcam(target_arr, classifHeatmapNorm)
keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2))
keras.utils.array_to_img(target_arr)

In [ ]:
image_f = 'Deforestation03HR_before_06_2018.jpg'

images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'
target_img = tf.keras.utils.load_img(images_path+image_f)
target_arr  = tf.keras.utils.img_to_array(target_img)

patches = get_patches(target_arr)
patches_predicts = trained_model.predict(resize_and_rescale(patches))
patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
patches_predicts_label = class_names[patches_predicts_id]
patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)
# classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

save_and_display_gradcam(target_arr, classifHeatmapNorm)
keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2))
keras.utils.array_to_img(target_arr)

In [ ]:
image_f = 'Deforestation01HR_after_06_2012.jpg'

images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'
target_img = tf.keras.utils.load_img(images_path+image_f)
target_arr  = tf.keras.utils.img_to_array(target_img)

patches = get_patches(target_arr)
patches_predicts = trained_model.predict(resize_and_rescale(patches))
patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
patches_predicts_label = class_names[patches_predicts_id]
patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)
# classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

save_and_display_gradcam(target_arr, classifHeatmapNorm)
keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2))
keras.utils.array_to_img(target_arr)

In [ ]:
image_f = 'Deforestation01HR_before_06_2011.jpg'

images_path = './../../data/MultiModalGenAI/deforestation/EldoradoNationalForest/'
target_img = tf.keras.utils.load_img(images_path+image_f)
target_arr  = tf.keras.utils.img_to_array(target_img)

patches = get_patches(target_arr)
patches_predicts = trained_model.predict(resize_and_rescale(patches))
patches_predicts_id = tf.math.argmax(patches_predicts, axis=-1)
patches_predicts_label = class_names[patches_predicts_id]
patches_predicts_forrest = np.clip(patches_predicts[:, np.where(class_names == 'forest')[0]], 0, None)

patch_num_y, patch_num_x = math.floor(target_arr.shape[0]/stride_height), math.floor(target_arr.shape[1]/stride_width)
# classifHeatmap = create_heatmap(patches_predicts_id, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmap = create_heatmap(patches_predicts_forrest, (patch_num_y, patch_num_x), (patch_height, patch_width))
classifHeatmapNorm =  np.uint8(255.*(classifHeatmap - classifHeatmap.min())/(classifHeatmap.max() - classifHeatmap.min()))

save_and_display_gradcam(target_arr, classifHeatmapNorm)
keras.utils.array_to_img( np.repeat(classifHeatmapNorm[:, :, np.newaxis], 3, axis=2))
keras.utils.array_to_img(target_arr)